## Predicting the Winners of European Soccer Matches

The goal of this assignment is to get some practice working with a somewhat messy relational database and building a multiclass classification model for complex data. Download the soccer database [here](https://www.kaggle.com/hugomathien/soccer) if you haven't already. Be sure to refer to the documentation on the website for information on what the variables in the dataset represent.


## Requirements

The 'deliverable' at the end of this assignment is a predictive model. The minimum requirements are simple:

1. **Clean/Preprocess the data, Select Features:** Clean and preprocess the data, then select features for inclusion. Your model must incorporate information from **at least one** table in addition to the 'match' table. 

1. **Split the data:** Split the processed dataset into a training set and a test set. The size of the test set should be no more than 20% of the size of the full dataset. Make sure that the test set consists of randomly selected rows from the dataset, not just the first or last 20% of the data. You can use a function from scikit-learn for this.

2. **Construct a model:** Using the training data ONLY, build the best model you can for predicting whether a given match is a home team win, an away team win, or a draw. Validate your model initially using $k$-fold cross-validation on the training set. Then, when you are satisfied with your results on the training set, make predictions on the test set and consider the precision and recall scores of your model. You can use sklearn's classification report to report these values.

3. **Discuss your results:** How did your model perform? Which features were most informative? Which were not? What might have helped to improve the quality of your results? 

Put all of the work that you would like me to see in this notebook. Use Markdown cells to explain the steps in your analysis. Include visualizations as appropriate. When you've completed the assignment, export a copy of the notebook to html by clicking File > Download As > HTML. Submit the html file only on myCourses by the due date.



## Submission & Due Date

Submit on myCourses by Tuesday, August 1, 11:59PM.

THIS FILE USES BETTING ODDS FROM THREE BETTING HOUSES AND ALL TWO WAY INTERACTIONS AMONG THEM. IT DOES NOT USE ANY INFORMATION FROM EACH TEAM.

In [20]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_curve, auc, confusion_matrix

# create database connection
conn = sqlite3.connect('C:/Analytics/DATA803/PythonII/PythonII_Assignments/soccer/database.sqlite')

# read the sql query and return a Pandas dataframe
# This reads meta-data from the SQLite database.  Each of these pieces
# of information are in separate tables.  We need to get the tables
#
pd.read_sql_query('select * from sqlite_master where type="table"', conn)

# This brings in 3 tables as 3 dataframes.
match = pd.read_sql_query('select * from Match', conn)
team_attributes = pd.read_sql_query('select * from Team_Attributes', conn)
team = pd.read_sql_query('select * from Team', conn)

# For each team grab random set of attributes regardless of dates.  
# Then shrink data frame.  SO NOW TEAM_ATTS CONTAINS OUR TEAM 
# ATTRIBUTES RATHER THAN THE DF TEAM_ATTRIBUTES
teams_w_atts = team_attributes['team_api_id'].unique()

unique_team_att_idx = [] #Create index for values selected at random.
for tm in teams_w_atts:
    idx = team_attributes[team_attributes['team_api_id'] == tm].index
    unique_team_att_idx.append(np.random.choice(idx, size=1)[0])
    
team_atts = team_attributes.loc[unique_team_att_idx, :]

In [2]:
# these are functions we need.
def change_column_name(dataframe=None, current=None, to=None):
    cols = list(dataframe.columns)
    for i, col in enumerate(cols):
        if col == current:
            cols[i] = to
        else:
            continue
    return cols
# This function will attach the "add" word to all variables to
# distinguish home from away variables that will be merged.
def change_all_column_names(dataframe=None,add=None):
    cols=list(dataframe.columns)
    for i,col in enumerate(cols):
        cols[i]=add+col
    return cols
# Now assign this to the team_attribute variable.

In [4]:
# This box retains only the team_atts variables that what we want
# and calculates the result of each match. Determine if a match
#was a draw(1), home win(2), an away win(3).
#match.drop(['id'], axis=1, inplace=True)
match['matchresult']=1
match['matchresult'] = np.where((['home_team_goal'] > \
            match['away_team_goal']),2,match['matchresult'])
match['matchresult'] = np.where((match['home_team_goal'] < \
            match['away_team_goal']),3,match['matchresult'])
# Name variables to keep.
match =match[['matchresult','home_team_api_id', 'away_team_api_id',\
          'B365H', 'B365D', 'B365A','IWH', 'IWD', 'IWA', \
             'BWH', 'BWD', 'BWA']]
team_atts.drop(['id', 'date'], axis=1, inplace=True)

# Also get rid of all player variables -- i.e., retain all variables
# that do not have 'play' in position 5:8.
#cols=[c for c in match.columns if c.lower()[5:9]!= 'play']
#match=match[cols]
# might as well datetime these
#match['date'] = pd.to_datetime(match['date'])

In [5]:
# This cell merges data from the team_atts and match databases.
# This calls the function change column names so we have the 
# same variable in both databases.  Then we merge MATCH with
# TEAM_ATTS on the field team_api_id.  Do Home first and then away.
# Create a list of variable names for TEAM_ATTS subsequent use.
origcols=team_atts.columns
cols = change_column_name(dataframe=match, current='home_team_api_id', \
            to='team_api_id')
match.columns = cols
# Add "home" to all of the team_atts attributes.
cols=change_all_column_names(dataframe=team_atts,add='home_')
team_atts.columns=cols
# 'home_' has been added to to the join field team_api_id. Get
# rid of it before the merge for the merge field.
cols = change_column_name(dataframe=team_atts, \
            current= 'home_team_api_id',to='team_api_id')
team_atts.columns = cols
# Now merge the match and team_attributes. Start with a clean
# dataframe.
match_w_home_atts = pd.DataFrame()
match_w_home_atts = match.merge(team_atts, on='team_api_id', how='left')
# Now change team_api_id back to home_team_api_id in match_w_home_atts
# and in match.
cols = change_column_name(dataframe=match_w_home_atts, current= \
        'team_api_id', to='home_team_api_id')
match_w_home_atts.columns = cols
cols = change_column_name(dataframe=match, current= \
        'team_api_id', to='home_team_api_id')
match.columns = cols
# Now restore the original names of TEAM_ATTS so HOME does not
# show up in the away attributes.
team_atts.columns = origcols

# Now do away team.
cols = change_column_name(dataframe=match_w_home_atts, current=\
            'away_team_api_id', to='team_api_id')
match_w_home_atts.columns = cols
# Add "away" to all of the team_atts attributes.
cols=change_all_column_names(dataframe=team_atts,add='away_')
team_atts.columns=cols
# 'away_' has been added to to the join field team_api_id. Get
# rid of it before the merge.
cols = change_column_name(dataframe=team_atts, current='away_team_api_id', \
            to='team_api_id')
team_atts.columns = cols
# Now merge the match and team_attributes.
match_w_homeaway_atts = match_w_home_atts.merge(team_atts, \
            on='team_api_id', how='left')
# Now change team_api_id back to away_team_api_id in new file and
# match.
cols = change_column_name(dataframe=match_w_homeaway_atts, current= \
        'team_api_id', to='away_team_api_id')
match_w_homeaway_atts.columns = cols
cols = change_column_name(dataframe=match_w_homeaway_atts, current= \
        'team_api_id', to='away_team_api_id')
# Now change team_atts back to the original and get rid of team_ids
# from match_w_homeaway_atts.
team_atts.columns = origcols
# Now get rid of home and away team identifiers and keep only betting
# house information to prepare the data for modelling.
match_w_homeaway_atts=match_w_homeaway_atts[['matchresult',\
            'B365H', 'B365D', 'B365A','IWH', 'IWD', 'IWA', \
             'BWH', 'BWD', 'BWA']]
#match_w_homeaway_atts.drop(['home_team_api_id','away_team_api_id', \
#            'home_team_fifa_api_id'], axis=1, inplace=True)

In [21]:
match_w_homeaway_atts.shape


(22502, 10)

In [22]:
# This cell eliminats missing values. It starts by eliminating
# columns that have more than THRESHOLD percentage of missing values.
# From the resulting dastaaframe, iIt then eliminates all rows
# having any missing values.
threshold=0.15
maxcols=match_w_homeaway_atts.shape[1]-1
for i in range(maxcols,0,-1):
    nummiss = match_w_homeaway_atts[match_w_homeaway_atts.columns[i]].isnull().sum()
    total=match_w_homeaway_atts.shape[0]
    if nummiss/total < threshold:
        continue
    print('Dropped: ',i,match_w_homeaway_atts.columns[i],nummiss,total,nummiss/total)
    match_w_homeaway_atts.drop(match_w_homeaway_atts.columns[i],axis=1,inplace=True)
# Now eliminate all rows having missing values.
match_w_homeaway_atts=match_w_homeaway_atts.dropna(axis=0)
match_w_homeaway_atts.shape

(22502, 10)

In [23]:
# This cell is for getting interactions and transformations.
#Now loop through the data frame for variables 2 [index=1] through
# n. In full implementation, the y variable will be in its own df
# that will get converted to an array for fitting the logistic
# regression. If a character variable is found, convert it to dummy
# variables.If a number is found, make it floating point. First set up
# df.
# NOTE: matchresult is the first column and it is never part of the
# interactions.  Start all loops at index 1 and not 0.
soccer=match_w_homeaway_atts
outsoccer=pd.DataFrame()
outsoccer=match_w_homeaway_atts[['matchresult']]
for i in range(1,soccer.shape[1]):
# Create an empty dataframe for filling with variable(s).
    newcol=pd.DataFrame()
#Test if this is a numeric variable. If so, make sure it is type float
# in the output file.        
    if(np.issubdtype(soccer[soccer.columns[i]], int)) or \
      (np.issubdtype(soccer[soccer.columns[i]], float)):
        newcol[0]=soccer[soccer.columns[i]].astype(float)
        col=soccer.columns[i]
        newcol.columns=[col]
        outsoccer=pd.concat([outsoccer,newcol],axis=1)
########### THIS IS THE NO TRANSFORM FILE.  DO NOT USE SQUARED OR
########### INVERSE TRANSFORMATIONS.
# Add a squared transformation.
#        newcol=pd.DataFrame()
#        newcol[0]=(soccer[soccer.columns[i]].astype(float))**2
#        col='sqrd'+soccer.columns[i]
#        newcol.columns=[col]
#        outsoccer=pd.concat([outsoccer,newcol],axis=1)
# Now add an inverse transformation. Add one before taking inverse
# to protect against zero division without substantially impacting
# model results.
#        newcol=pd.DataFrame()
#        newcol[0]=1/((soccer[soccer.columns[i]].astype(float)+1))
#        col='inv'+soccer.columns[i]
#        newcol.columns=[col]
#        outsoccer=pd.concat([outsoccer,newcol],axis=1)
############# INCLUDE THE CODE ABOVE IF WE WANT TRANSFORMS
# If it is not a numeric variable, make dummy variables and
# append them to the output file.
    else:
        newcol=pd.get_dummies(soccer[soccer.columns[i]],drop_first=True)
        cols=change_all_column_names(dataframe=newcol,add=\
                soccer.columns[i])
        newcol.columns=cols
        outsoccer=pd.concat([outsoccer,newcol],axis=1)
# Now that we have all base variables, calculate all two-way
# interactions and append them to the file.  Note all variables
# in the file are now numeric. Because I will concatentate to the
# outsoccer file, the number of columns will increase. Create a
# variable for looping that will control the number of columns addressed.
maxcols=outsoccer.shape[1]
for i in range(1,maxcols-1):
    for j in range(i+1,maxcols):
# Create empty dataframe for each interaction.
        newcol=pd.DataFrame()
# Multiply each column by all columns -- but duplication and squaring
# is avoided through the indexing.
        newcol[0]=outsoccer[outsoccer.columns[i]]* \
            outsoccer[outsoccer.columns[j]]
        col=outsoccer.columns[i]+outsoccer.columns[j]
        newcol.columns=[col]
# Do not write columns full of zeroes resulting from the 
# multiplication of dummy variables from the same original
# categorical variable. Also do not write a variable that
# is the multiplication of a variable squared by itself 
# (thereby making a cubed variable which we do not want),
# of a variable that is a variable multiplied by its inverse.
        if(max(newcol.ix[:,0])> 0) and \
            sum(newcol.ix[:,0]) != sum(outsoccer.ix[:,i]**3):
            outsoccer=pd.concat([outsoccer,newcol],axis=1)
# In case calculations caused problems, drop all rows with 
# missing values.
outsoccer=outsoccer.dropna()

In [13]:
outsoccer.shape

(22502, 46)

In [24]:
# Prep the arrays to fit the data.
y=np.array(outsoccer['matchresult'])
y

outsoccer.drop(['matchresult'],axis=1,inplace=True)
X=outsoccer.loc[:,:].as_matrix()

# Now split the data into training and test.
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=0)
                      

In [10]:
y

array([1, 1, 3, ..., 2, 1, 2], dtype=int64)

In [25]:
# Now fit the multinomial regression model (lbfgs.)
#lmsk = LogisticRegression(solver='lbfgs', \
#        multi_class='multinomial').fit(Xtr, ytr)
lmsk = LogisticRegressionCV(solver='lbfgs',cv=8, \
        multi_class='multinomial').fit(Xtr, ytr)
preds = lmsk.predict(X)
#lm.coef_
print(classification_report(y, preds))

             precision    recall  f1-score   support

          1       0.45      0.00      0.00      5686
          2       0.54      0.87      0.67     10340
          3       0.51      0.47      0.49      6476

avg / total       0.51      0.53      0.45     22502



In [26]:
# Confusion matrix for the sklearn model and test data.
predslmsk=lmsk.predict(Xte)
ppredslmsk=lmsk.predict_proba(Xte)
cnfmtrx=pd.DataFrame(confusion_matrix(yte,predslmsk))
print(cnfmtrx)

   0     1    2
0  1   859  298
1  0  1774  269
2  1   675  624


In [17]:
# Get the overall accuracy, precision, and recall (for test data).
acc=accuracy_score(yte, predslmsk)
prec=cnfmtrx.iat[1,1]/(cnfmtrx.iat[1,1]+cnfmtrx.iat[0,1])
rec=cnfmtrx.iat[1,1]/(cnfmtrx.iat[1,1]+cnfmtrx.iat[1,0])
print(acc, prec, rec)

0.532992668296 0.673756171667 1.0


In [28]:
# Close the connection
conn.close()